In [1]:
import paho.mqtt.client as mqtt
import json
import redis

In [2]:
CLIENT_ID = 's291462'

In [3]:
REDIS_HOST = 'redis-19466.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 19466
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'iIQjppRdJ7f3kKU4Ng5IDkgv6TvaPnrN'
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [4]:
client = mqtt.Client()
def on_connect(client, userdata, flags, rc):
    # rc is the result code print to check status
    client.subscribe(CLIENT_ID)
    print(f'Connected with result code {str(rc)}')

In [5]:
def on_message(client, userdata, msg):
    # extracting message from msg byte format to string
    message = msg.payload.decode()
    topic = msg.topic
    # converting json string to dict
    str_to_dict = json.loads(message)
    mac_address = str_to_dict['mac_address']
    battery_level = str_to_dict['battery_level']
    power_plugged = int(str_to_dict['power_plugged']) # exercise require to have
    timestamp_ms = str_to_dict['timestamp']

    # Adding data to redis 
    mac_battery = mac_address + ':battery'
    mac_power = mac_address + ':power'
    # Adding time series if does not exists
    try:
        redis_client.ts().create(mac_battery)
        redis_client.ts().create(mac_power)
    except redis.ResponseError:
        pass
    redis_client.ts().add(mac_battery, timestamp_ms, battery_level)
    redis_client.ts().add(mac_power, timestamp_ms, power_plugged)
    print(f"Received message '{message}' on topic {topic}")

In [6]:
# Set the callbacks
client.on_connect = on_connect
client.on_message = on_message
client.connect('mqtt.eclipseprojects.io', 1883)
client.loop_forever()

Connected with result code 0
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492254623, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492255628, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492256635, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492257638, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492258642, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492259646, "battery_level": 67, "power_plugged": false}' on topic s291462
Received message '{"mac_address": "0xacde48001122", "timestamp": 1674492260648, "battery_level": 67, "power_plugg

KeyboardInterrupt: 